In [1]:
require(tidyverse)

필요한 패키지를 로딩중입니다: tidyverse

─ Attaching packages ──────────────────── tidyverse 1.3.1 ─

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

─ Conflicts ───────────────────── tidyverse_conflicts() ─
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [34]:
log(2132.486)

[1] 7.665044

In [48]:
df <- read_csv('../stage1/input/forSurv.csv')

df <- df %>% select(c(1,2,3,4))

beta <- c(0.1, 0.2)

# input : data, beta

loss_function <- function(data, beta){
    # calculate loss. divided into two sections. Left term and the Right term
    ordered_df <- data %>% arrange(by=time)
    timeY <- ordered_df %>% select(c(time, Y))
    Xdata <- ordered_df %>% select(-c(time,Y))
    
    # calculate left term
    left <- exp((Xdata %>% as.matrix()) %*% beta)
    left <- timeY %>% cbind(exp = left)
    left <- left %>%
        group_by(time) %>%
        summarise(
            exp_sum = sum(exp),
            log_exp_sum = log(exp_sum),
            n = n(),
            mul_with_n = n * log_exp_sum
        )

    left_term <- left$mul_with_n %>% sum()

    # calculate right term
    death_data <- ordered_df %>% filter(Y == 1)
    timeY <- death_data %>% select(c(time, Y))
    Xdata <- death_data %>% select(-c(time, Y))

    right <- ((Xdata %>% as.matrix()) %*% beta)
    right <- timeY %>% cbind(exp_log = right)

    right_term <- right$exp_log %>% sum()

    # calculate loss
    loss <- left_term - right_term
    return(loss)
}


ordered_df<- df %>% arrange(by=OVRL_SRVL_DTRN_DCNT)
timeY <- ordered_df %>% select(c(time=2,Y=1))
Xdata <- ordered_df %>% select(c(3:ncol(ordered_df)))
ordered_df <- cbind(timeY, Xdata)

# calculate left term
left <- exp((Xdata %>% as.matrix()) %*% beta)
left <- timeY %>% cbind( exp = left)
left <- left %>%
    group_by(time) %>%
    summarise(exp_sum = sum(exp),
    log_exp_sum = log(exp_sum),
    n=n(),
    mul_with_n = n*log_exp_sum)

left_term <- left$mul_with_n %>% sum

# calculate right term
death_data <- ordered_df %>% filter(Y==1)
timeY <- death_data %>% select(c(time,Y))
Xdata <- death_data %>% select(-c(time,Y))

right <- ((Xdata %>% as.matrix()) %*% beta)
right <- timeY %>% cbind(exp_log=right)

right_term <- right$exp_log %>% sum

loss <- left_term - right_term

loss

Rows: 10276 Columns: 16

─ Column specification ────────────────────────────
Delimiter: ","
dbl (16): BSPT_SRV, OVRL_SRVL_DTRN_DCNT, BSPT_IDGN_AGE, BSPT_SEX_CD, C18, C1...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



[1] 147135.8